In [20]:
import pandas as pd
import os
import time

<br>

### Find right datasets in each wave

In each folder associated to one wave, we have multiple datasets. However, not all datasets represent questionnaires but those are in minority. Since the goal of this notebook is to concat all questionnaires of each wave in its own dataset, we don't want to merge these data.

The way we decide to define which dataset to keep or not is to define a dictionnary with wave as keys and the number of people who have fill the questionnaire as values. This way, we can *automatically* detect if a dataset is a questionnaire or not by testing if the number of rows match the number of respondants.

`len(df)==wave_to_rows[f"wave{i}"]`

**Example**

The 8th wave has 46733 respondants. When iterating over each file from the wave 8 folder, we open each file, count the number of rows (`len(df)`), get the number of respondants in this wave (`wave_to_rows[f"wave{i}"]`) and test if these are the same values. In most cases, it's supposed to be the case (most datasets are questionnaires!) but when it's not, we just pass and go to the next dataset.

**Limitations**

The number of respondants is manually defined. A more scalable way to do it would have be to count the number of rows each dataset of each wave, get the *mode* (most present value) of the number of rows per wave and consider this value as the number of respondants. However, this also has some limitations (no guaranty of getting the right number).

In [21]:
wave_to_rows = {
    "wave1": 30419,
    "wave2": 37143,
    "wave3": 28463,
    "wave4": 58000,
    "wave5": 66065,
    "wave6": 68085,
    "wave7": 77202,
    "wave8": 46733    
}

### Directory structure

Directory with data: `data`.

In this directory, I have multiple folders: one for each wave, unzipped. Each folder contains all the dataset for the wave. No name have been change (folders or files).


In [22]:
start = time.time()

# iterate over each folder
root_dir = '../../data'
for folder in os.listdir(root_dir):
    
    # init parameters for the current wave
    merged_df = pd.DataFrame()
    nfiles = 0
    
    # get folder path
    folder_path = os.path.join(root_dir, folder)
    if os.path.isdir(folder_path) and folder != 'concat':
        
        # 6th element of folder name is in fact wave number
        i = folder[6]

        # iterate over each file in the folder
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            
            # check if it's dataset
            if file_path.endswith('dta'):
                
                try:
                    df = pd.read_stata(file_path)
                    
                    # test number of rows
                    rows_expected = wave_to_rows[f"wave{i}"]
                    actual_rows = len(df)
                    if rows_expected==actual_rows:

                        df = df.add_prefix(file.split('.')[0] + '_')

                        # merge dfs
                        merged_df = pd.merge(merged_df, df, left_index=True,
                                             right_index=True, how='outer')
                        nfiles += 1
                
                # avoid non-unique error
                except ValueError:
                    pass
                
        # save df
        merged_df.to_csv(f'../../data/concat/concatwave{i}.csv.gz', index=False)
        print(f"Wave {i} (with {nfiles} files) concatenated and saved!\n")

print(f"\n\nTime (sec): {(time.time()-start):.2f}")

Wave 1 (with 26 files) concatenated and saved!

Wave 4 (with 27 files) concatenated and saved!

Wave 5 (with 27 files) concatenated and saved!

Wave 3 (with 15 files) concatenated and saved!

Wave 6 (with 30 files) concatenated and saved!

Wave 8 (with 31 files) concatenated and saved!

Wave 7 (with 35 files) concatenated and saved!

Wave 2 (with 26 files) concatenated and saved!



Time (sec): 2625.53
